### This make sure that encodec caps can be used to reconstructe WAV

In [4]:
from encodec import EncodecModel
from encodec.utils import convert_audio

import torchaudio
import torch
from encodec.compress import compress, decompress, decompress_from_frames
import json

In [2]:
model = EncodecModel.encodec_model_24khz()
# model.set_target_bandwidth(6.0) # => [1, 8, T]
# model.set_target_bandwidth(3.0) # => [1, 4, T]
model.set_target_bandwidth(1.5) # => [1, 2, T]

In [74]:
en_1_caps = "../data/processed/encodec1/TEST_CAPTIONS_coco_1_cap_per_img_1_min_word_freq.json"
en_2_caps = "../data/processed/encodec2/TEST_CAPTIONS_coco_1_cap_per_img_1_min_word_freq.json"
en_1_wordmap = "../data/processed/encodec1/WORDMAP_coco_1_cap_per_img_1_min_word_freq.json"
en_2_wordmap = "../data/processed/encodec2/WORDMAP_coco_1_cap_per_img_1_min_word_freq.json"

In [75]:
with open(en_1_caps, "rb") as f:
    en_1_caps = json.load(f)
with open(en_2_caps, "rb") as f:
    en_2_caps = json.load(f)
with open(en_1_wordmap, "rb") as f:
    en_1_wordmap = json.load(f)
with open(en_2_wordmap, "rb") as f:
    en_2_wordmap = json.load(f)

In [29]:
en_1_wordmap_rev = {value: key for key, value in en_1_wordmap.items()}
en_2_wordmap_rev = {value: key for key, value in en_2_wordmap.items()}

In [69]:
cap_0_en_1 = [en_1_wordmap_rev[cap] for cap in en_1_caps[10] if en_1_wordmap_rev[cap] != "<pad>" and en_1_wordmap_rev[cap] != "<start>" and en_1_wordmap_rev[cap] != "<end>"]
cap_0_en_2 = [en_2_wordmap_rev[cap] for cap in en_2_caps[10] if en_2_wordmap_rev[cap] != "<pad>" and en_2_wordmap_rev[cap] != "<start>" and en_2_wordmap_rev[cap] != "<end>"]

In [70]:
cap_0_en_1 = list(map(lambda x: int(x), cap_0_en_1))
cap_0_en_2 = list(map(lambda x: int(x), cap_0_en_2))

In [71]:
codes = torch.as_tensor([cap_0_en_1,cap_0_en_2])

In [72]:
codes = codes.unsqueeze(0)

In [73]:
wav, sr = decompress_from_frames(model, codes)
wav.numpy()
import IPython.display as ipd
display(ipd.Audio(wav, rate = 24000))